<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/Tech16_HW5_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install "crewai[tools]"

In [28]:
!pip install llama-index --upgrade

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

###Retrieve and write the GS market outlook to a pdf

In [4]:
!wget "https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf" -O GS_SnP500.pdf

--2025-03-05 05:02:25--  https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf
Resolving privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)... 23.192.230.149, 23.192.230.134
Connecting to privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)|23.192.230.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘GS_SnP500.pdf’

GS_SnP500.pdf           [  <=>               ]   6.88M  26.3MB/s    in 0.3s    

2025-03-05 05:02:26 (26.3 MB/s) - ‘GS_SnP500.pdf’ saved [7209986]



###Retrieve and write the JPM market outlook to a pdf

In [5]:
!wget "https://am.jpmorgan.com/content/dam/jpm-am-aem/global/en/2025%20Year-Ahead%20Investment%20Outlook.pdf" -O JPM_Outlook.pdf

--2025-03-05 05:02:26--  https://am.jpmorgan.com/content/dam/jpm-am-aem/global/en/2025%20Year-Ahead%20Investment%20Outlook.pdf
Resolving am.jpmorgan.com (am.jpmorgan.com)... 146.143.7.3
Connecting to am.jpmorgan.com (am.jpmorgan.com)|146.143.7.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘JPM_Outlook.pdf’

JPM_Outlook.pdf         [     <=>            ]   1.44M  1.36MB/s    in 1.1s    

2025-03-05 05:02:27 (1.36 MB/s) - ‘JPM_Outlook.pdf’ saved [1512586]



In [6]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI


reader = SimpleDirectoryReader(input_files=["GS_SnP500.pdf"])
docs = reader.load_data()
reader = SimpleDirectoryReader(input_files=["JPM_Outlook.pdf"])
docs2 = reader.load_data()

In [7]:
docs[117].get_content()

'Goldman Sachs\nAmsterdam\nAtlanta\nAustin\nBeijing\nBoston\nChicago\nDallas\nDenver\nDetroit\nDubai\nDublin\nFrankfurt\nGeneva\nHong Kong\nHouston\nLondon\nLos Angeles\nLuxembourg\nMadrid\nMelbourne\nMiami\nMilan\nMonaco\nNashville / Brentwood\nNew York\nParis\nPhiladelphia\nSan Francisco\nSeattle\nShanghai\nShenzhen\nSingapore\nStockholm\nSydney\nTel Aviv\nWashington, D.C.\nWest Palm Beach\nZurich\nwww.gs.com'

In [8]:
# print(type(docs))
# print(len(docs))

<class 'list'>
118


In [9]:
# print(type(docs2))
# print(len(docs2))

<class 'list'>
16


In [48]:
# from crewai_tools import SerperDevTool, WebsiteSearchTool, LlamaIndexTool

# llm = OpenAI(model="gpt-3.5-turbo")
# index = VectorStoreIndex.from_documents(docs)
# index = VectorStoreIndex.from_documents(docs2)
# query_engine = index.as_query_engine(similarity_top_k=5,llm=llm)

/usr/local/lib/python3.11/dist-packages/llama_index/core/indices/base.py:110: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)


In [13]:
# #Not used
# query_tool = LlamaIndexTool.from_query_engine(
#     query_engine,
#     name="Global Outlook Query Tool",
#     description="Use this tool to lookup the 2025 global finance outlook of GS and JPM"
# )

In [14]:
query_tool.args_schema.schema()

<ipython-input-14-fe53991c08af>:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  query_tool.args_schema.schema()


{'description': 'Schema for query tool.',
 'properties': {'query': {'description': 'Search query for the query tool.',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'QueryToolSchema',
 'type': 'object'}

In [15]:
%pip install llama-index-vector-stores-weaviate

In [16]:
!pip install llama-index-vector-stores-chroma

In [18]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.get_or_create_collection("tech16HW5example")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [19]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./").load_data("GS_SnP500.pdf")
documents = SimpleDirectoryReader("./").load_data("JPM_Outlook.pdf")
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query("What is the 2025 outlook for US GDP?")
print(response)

Loading files: 100%|██████████| 2/2 [00:09<00:00,  4.79s/file]
/usr/local/lib/python3.11/dist-packages/llama_index/core/indices/base.py:110: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)


The 2025 outlook for US GDP is an expansion of 2.3%.


In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = index.as_retriever()
query_chromadb = RetrieverQueryEngine(retriever=retriever)

llama_chroma_tool = LlamaIndexTool.from_query_engine(
    query_chromadb,
    name="Global Outlook Query Tool",
    description="Use this tool to lookup the 2025 global finance outlook of GS and JPM"
)



### Include critic using Sequential and using tools to search the web and search the chromadb vector database with the JPM and GS pdfs

In [23]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# --------------------------
# Configuration for a generic research task
# --------------------------
topic = "What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?"

critic_description = (
    f"Review the financial analysis report on the outlook of the S&P 500 and summarizing the findings on '{topic}'. "
    "Provide detailed feedback on the clarity, coherence, and overall quality of the report, "
    "including strengths, weaknesses, and suggestions for improvement."
)
critic_expected_output = (
    "A collection of bullet points or paragraphs providing constructive critique of the analysis report, "
    "highlighting strengths, weaknesses, and potential improvements."
)

rewriter_description = (
    f"Rewrite the analysis report on '{topic}' by incorporating the feedback from the Critic. "
    "Produce a refined version that addresses the criticisms and improves clarity and structure."
)
rewriter_expected_output = (
    f"A refined markdown analysis report on '{topic}', incorporating the critic's feedback, "
    "with clear headings and detailed explanations."
)

# # --------------------------
# # Set up the tools for the Researcher agent: a web search tool and a web content reader.
# # --------------------------
# search_tool = SerperDevTool()         # Enables web searching via the Serper API
# web_reader_tool = WebsiteSearchTool()   # Retrieves and reads content from web pages

# Create the Researcher agent whose role is to gather info from the web and PDFs
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), llama_chroma_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)


# Create the Critic agent
critic_agent = Agent(
    name="Financial Analysis Critic",
    role="Financial Content Critic",
    goal="Review and provide constructive feedback on the analysis report",
    backstory="An expert in critically evaluating financial content to ensure quality, accuracy, and clarity.",
    tools=[],  # No special tools needed for this critique task
    verbose=True
)


# Create the Rewriter agent
rewriter_agent = Agent(
    name="Rewriter",
    role="Content Editor",
    goal="Rewrite the analysis report by incorporating the critic's feedback",
    backstory="A skilled editor who refines content to enhance clarity, coherence, and overall quality.",
    tools=[],  # No special tools needed for this rewriting task
    verbose=True
)

# Define tasks
research_task = Task(
    description=f"Conduct a comprehensive analysist of '{topic}' and summarize key findings and risk factors in 2025.",
    expected_output=f"Full analysis report in bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Using the insights provided, develop an engaging and structured report on '{topic}' that highlights the headwinds the S&P 500 index faces using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]
)

# Define the critic task for the Critic agent.
critic_task = Task(
    description=critic_description,
    expected_output=critic_expected_output,
    agent=critic_agent,
    context=[write_task]  # This task uses the output of the write_task as context
)

# Define the rewriting task for the Rewriter agent.
rewriter_task = Task(
    description=rewriter_description,
    expected_output=rewriter_expected_output,
    agent=rewriter_agent,
    context=[research_task, write_task, critic_task]  # This task uses both the original report and the critique as context
)

# --------------------------
# Assemble the crew with all four agents and tasks.
# --------------------------
research_crew = Crew(
    agents=[researcher, writer, critic_agent, rewriter_agent],
    tasks=[research_task, write_task, critic_task, rewriter_task],
    process=Process.sequential,  # Executes tasks in sequence: research -> writing -> critique -> rewrite
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))


# Agent: Financial Research Analyst
## Task: Conduct a comprehensive analysist of 'What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?' and summarize key findings and risk factors in 2025.


# Agent: Financial Research Analyst
## Thought: I need to gather information about the 2025 outlook for the S&P 500 index from both JPMorgan (JPM) and Goldman Sachs (GS) to conduct a comprehensive analysis. I'll use the Global Outlook Query Tool to obtain this information.
## Using tool: Global Outlook Query Tool
## Tool Input: 
"{\"query\": \"2025 outlook for S&P 500 index site:goldmansachs.com OR site:jpmorgan.com\"}"
## Tool Output: 
The 2025 outlook for the S&P 500 index is discussed in the document from Goldman Sachs. It mentions that while the investment themes of US Preeminence and Stay Invested remain valid, the expectation is that US equities may not continue to outperform by the same magnitude as they have in the past 

# Outlook for the S&P 500 in 2025: A Comparative Analysis of Market Views from Goldman Sachs and JPMorgan Chase

## Overall Market View

### Goldman Sachs (GS)
Goldman Sachs presents a nuanced perspective on the future of the S&P 500. Their key themes suggest:

- **US Preeminence and "Stay Invested" Approach**: GS advocates for maintaining investment in US equities, emphasizing that while the market has demonstrated substantial growth in the past, particularly over the last 16 years, such outsized returns may not be replicated in the near future. They anticipate more tempered gains that align closer with historical averages. 

- **Caution on Potential Headwinds**: GS identifies various economic and geopolitical risks that could substantially hinder future performance. They emphasize the importance of recognizing these factors and adopting a more cautious approach to investing, indicating a probable moderation in growth relative to what investors may expect based on past performance.

### JPMorgan Chase (JPM)
Conversely, JPMorgan expresses a more bullish outlook concerning the S&P 500's potential for 2025, supported by several key factors:

- **Positive Outlook for Further Gains**: JPM argues for a bright future for the index, primarily driven by a broadening participation among stocks in the ongoing market rally. This suggests that the current growth is supported by multiple companies rather than a concentration in just a few market leaders, which traditionally signals a healthier market landscape.

- **Historical Trends Favoring Gains**: By referencing periods of low market volatility alongside historical performance metrics following previous all-time highs, JPM provides evidence that supports their optimistic forecast for sustained growth in the S&P 500 index.

- **Acknowledgment of Risks**: While JPM does recognize potential risks that could impact the market—such as economic uncertainties and geopolitical factors—their overall sentiment remains constructive. They envision a steady ascent in stock prices, bolstered by positive market dynamics.

## Key Findings
Synthesizing insights from both firms reveals several critical points:

- **Contrasting Emphases on Risks and Growth**: Both institutions present fundamentally positive outlooks for the S&P 500; however, Goldman Sachs emphasizes possible risks and moderated performance potential, whereas JPMorgan emphasizes the strength and breadth of the market rally, suggesting a more robust growth trajectory moving forward.

- **Outlook for Moderate vs. Aggressive Gains**: GS hints at a conservative forecast in light of expected performance adjustments, while JPM exhibits confidence in continued upward movement in stock prices, potentially leading to more aggressive gains.

## Risk Factors for 2025
Both firms outline several external challenges that could affect the S&P 500's trajectory through 2025:

- **Economic Uncertainties**: Concerns about fluctuations in consumer spending patterns and corporate profits remain central, as shifting economic conditions could adversely impact market performance.

- **Geopolitical Tensions**: Global political instabilities pose risks that could disrupt market stability and investor confidence.

- **Monetary Policy Changes**: Anticipated changes in interest rates by the Federal Reserve could significantly affect investment landscapes and liquidity in the market, altering the patterns of growth.

- **Market Volatility**: Various unforeseen global events could trigger significant market shifts, leading to heightened volatility, which would affect the S&P 500’s performance.

## Conclusion
In summary, both Goldman Sachs and JPMorgan Chase forecast a generally positive outlook for the S&P 500 through 2025; however, they arrive at this conclusion from contrasting viewpoints. JPMorgan asserts a more optimistic growth potential, underpinned by a broad participation of securities in the rally, while Goldman Sachs advocates a more cautious strategy, expecting moderated returns compared to the significant gains observed previously.

Investors and strategists must carefully weigh these distinct perspectives alongside the articulated risks when formulating their investment strategies for the upcoming years. Understanding these contrasting views is vital for making informed decisions in an evolving market landscape. Additionally, given the potential for volatility and risk, it might be prudent for investors to consider diversified portfolios that can withstand fluctuations while capturing growth opportunities in the S&P 500. 

In terms of actionable strategies, decision-makers may focus on sectors showing strength in participation according to JPMorgan’s analysis, while also being vigilant of the cautionary signals raised by Goldman Sachs to avoid overexposure.

###Include Critic Agent with Hierarchical process using tools to search the web and search the chromadb vector database with the JPM and GS pdfs

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

# --------------------------
# Configuration for a generic research task
# --------------------------
topic = "What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?"

critic_description = (
    f"Review the financial analysis report on the outlook of the S&P 500 and summarizing the findings on '{topic}'. "
    "Provide detailed feedback on the clarity, coherence, and overall quality of the report, "
    "including strengths, weaknesses, and suggestions for improvement."
)
critic_expected_output = (
    "A collection of bullet points or paragraphs providing constructive critique of the analysis report, "
    "highlighting strengths, weaknesses, and potential improvements."
)

rewriter_description = (
    f"Rewrite the analysis report on '{topic}' by incorporating the feedback from the Critic. "
    "Produce a refined version that addresses the criticisms and improves clarity and structure."
)
rewriter_expected_output = (
    f"A refined markdown analysis report on '{topic}', incorporating the critic's feedback, "
    "with clear headings and detailed explanations."
)

# # --------------------------
# # Set up the tools for the Researcher agent: a web search tool and a web content reader.
# # --------------------------
# search_tool = SerperDevTool()         # Enables web searching via the Serper API
# web_reader_tool = WebsiteSearchTool()   # Retrieves and reads content from web pages

# Create the Researcher agent whose role is to gather info from the web and PDFs
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), llama_chroma_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)


# Create the Critic agent
critic_agent = Agent(
    name="Financial Analysis Critic",
    role="Financial Content Critic",
    goal="Review and provide constructive feedback on the analysis report",
    backstory="An expert in critically evaluating financial content to ensure quality, accuracy, and clarity.",
    tools=[],  # No special tools needed for this critique task
    verbose=True
)


# Create the Rewriter agent
rewriter_agent = Agent(
    name="Rewriter",
    role="Content Editor",
    goal="Rewrite the analysis report by incorporating the critic's feedback",
    backstory="A skilled editor who refines content to enhance clarity, coherence, and overall quality.",
    tools=[],  # No special tools needed for this rewriting task
    verbose=True
)

# Define tasks
research_task = Task(
    description=f"Conduct a comprehensive analysist of '{topic}' and summarize key findings and risk factors in 2025.",
    expected_output=f"Full analysis report in bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Using the insights provided, develop an engaging and structured report on '{topic}' that highlights the headwinds the S&P 500 index faces using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]
)

# Define the critic task for the Critic agent.
critic_task = Task(
    description=critic_description,
    expected_output=critic_expected_output,
    agent=critic_agent,
    context=[write_task]  # This task uses the output of the write_task as context
)

# Define the rewriting task for the Rewriter agent.
rewriter_task = Task(
    description=rewriter_description,
    expected_output=rewriter_expected_output,
    agent=rewriter_agent,
    context=[research_task, write_task, critic_task]  # This task uses both the original report and the critique as context
)

# Set up the LLM-based manager for the hierarchical process.
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# Assemble the crew with all four agents and tasks.
research_crew = Crew(
    agents=[researcher, writer, critic_agent, rewriter_agent],
    tasks=[research_task, write_task, critic_task, rewriter_task],
    process=Process.hierarchical,  # Executes tasks using hte hierarchical process
    manager_llm=manager_llm,         # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))


<ipython-input-25-62c9f14c7cf1>:113: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  manager_llm = ChatOpenAI(temperature=0, model="gpt-4")


# Agent: Crew Manager
## Task: Conduct a comprehensive analysist of 'What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?' and summarize key findings and risk factors in 2025.


# Agent: Crew Manager
## Thought: First, I need to gather information about the 2025 outlook for the S&P 500 index from the overall market perspective. Then, I will need to find the views of JPM and GS on the same topic. I can use the 'Search the internet with Serper' tool to find this information. After gathering the necessary data, I can delegate the task of comparing and contrasting these views to the Financial Research Analyst.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"2025 outlook for the S&P 500 index market view\"}"
## Tool Output: 
{'searchParameters': {'q': '2025 outlook for the S&P 500 index market view', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Stock Market 